<a href="https://colab.research.google.com/github/GutemaB2015/Gutema-thesis/blob/main/DDOSMitigation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import random

import time
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

DatasetUrl = 'https://raw.githubusercontent.com/GutemaB2015/DDOS-Detection-and-Mitigation/main/FillDataset.csv'
data = pd.read_csv(DatasetUrl)


label_encoder = preprocessing.LabelEncoder()
data['Label'] = label_encoder.fit_transform(data['Label'])

# Extraxt the features you want to base the foundation of our model training
features = ['Max_Packet_Length','Fwd_Packet_Length_Max','Flow_Packets_Sec','Flow_Bytes_Sec','Packet_Length_Std'
           ,'Packet_Length_Variance','Flow_IAT_Max','Fwd_IAT_Max','Subflow_Fwd_Bytes','Fwd_Packet_Length_Std'
           ,'Bwd_Packets_Sec','min_seg_size_forward','Init_Win_bytes_backward','Average_Packet_Size'
           ,'Packet_Length_Mean','Fwd_IAT_Total','Flow_IAT_Std','Fwd_IAT_Std','Avg_Fwd_Segment_Size'
           ,'Fwd_Packet_Length_Mean','Fwd_Header_Length','Fwd_IAT_Mean','Flow_IAT_Mean','Idle_Max'
           ,'Idle_Mean','Fwd_Packets_Sec']

# the packet of request to be sent for demonstration
new_traffic = [353,353,0.207573889,9.578052299,114.1646586,13033.56926,
               79000000,79000000,969,119.0835509,0.009884471,0,29200
               ,46.14285714,44.04545455,101000000,18000000,18400000
               ,48.45,48.45,728,5323696.105,5058439.7,79000000,20500000,0.197689418]

# Replace missing values with the mean of the column

data['Idle_Mean'].fillna(data['Idle_Mean'].median(), inplace=True)
data['Idle_Max'].fillna(data['Idle_Max'].median(), inplace=True)

data['Flow_IAT_Mean'].fillna(data['Flow_IAT_Mean'].median(), inplace=True)
data['Flow_IAT_Max'].fillna(data['Flow_IAT_Max'].median(), inplace=True)
data['Flow_IAT_Std'].fillna(data['Flow_IAT_Std'].median(), inplace=True)

data['Fwd_IAT_Total'].fillna(data['Fwd_IAT_Total'].median(), inplace=True)
data['Fwd_IAT_Mean'].fillna(data['Fwd_IAT_Mean'].median(), inplace=True)
data['Fwd_IAT_Std'].fillna(data['Fwd_IAT_Std'].median(), inplace=True)
data['Fwd_IAT_Max'].fillna(data['Fwd_IAT_Max'].median(), inplace=True)

data['Fwd_Packet_Length_Std'].fillna(data['Fwd_Packet_Length_Std'].median(), inplace=True)
data['Fwd_Packet_Length_Max'].fillna(data['Fwd_Packet_Length_Max'].median(), inplace=True)
data['Fwd_Packet_Length_Mean'].fillna(data['Fwd_Packet_Length_Mean'].median(), inplace=True)
data['Fwd_Packets_Sec'].fillna(data['Fwd_Packets_Sec'].median(), inplace=True)


data['Packet_Length_Variance'].fillna(data['Packet_Length_Variance'].median(), inplace=True)
data['Packet_Length_Std'].fillna(data['Packet_Length_Std'].median(), inplace=True)
data['Packet_Length_Mean'].fillna(data['Packet_Length_Mean'].median(), inplace=True)

data['Flow_Bytes_Sec'].fillna(data['Flow_Bytes_Sec'].median(), inplace=True)
data['Flow_Packets_Sec'].fillna(data['Flow_Packets_Sec'].median(), inplace=True)

data['Subflow_Fwd_Bytes'].fillna(data['Subflow_Fwd_Bytes'].median(), inplace=True)
data['Fwd_Header_Length'].fillna(data['Fwd_Header_Length'].median(), inplace=True)
data['Avg_Fwd_Segment_Size'].fillna(data['Avg_Fwd_Segment_Size'].median(), inplace=True)
data['Max_Packet_Length'].fillna(data['Max_Packet_Length'].median(), inplace=True)

data['Average_Packet_Size'].fillna(data['Average_Packet_Size'].median(), inplace=True)
data['Init_Win_bytes_backward'].fillna(data['Init_Win_bytes_backward'].median(), inplace=True)
data['min_seg_size_forward'].fillna(data['min_seg_size_forward'].median(), inplace=True)
data['Bwd_Packets_Sec'].fillna(data['Bwd_Packets_Sec'].median(), inplace=True)

def detectMitigate(requestTraffic):
    # Split the dataset into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(data[features], data["Label"], test_size=0.2, random_state=1)


    clf = GaussianNB()
    clf.fit(X_train.values, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Model Accuracy:", accuracy)

    prediction = clf.predict(requestTraffic)

    if prediction == 0:
      print("The traffic is not a DDoS attack.")
    else:
      # Create a list of blocked IP addresses
      blocked_ips = []

      if requestTraffic not in blocked_ips:
        # Add the DDOS traffic request to the blocked list
        blocked_ips.append(requestTraffic)
        print("a DDoS attack with accuracy percentage of : ",accuracy)

        with open("blocked_ips.txt", "w") as f:
          for requestTraffic in blocked_ips:
            f.write(str(requestTraffic )+ "\n")
      else:
         print("This request is already blocked.")



def sendRequest():
  numbers = []
  for i in range(26):
    number = random.randint(1, 100)
    if random.random() > 0.5:
      number = float(number)
    numbers.append(number)
  return numbers


# generate and send traffic requests until it is stopped
traffic = np.array(sendRequest())
new_traffic = np.array(new_traffic)




while True:
  print("Request Packet : " , sendRequest())
  detectMitigate(traffic.reshape(1,-1))

  character = input()
  if character == "x":
    print("Packet Sending stopped:")
    time.sleep(1)
    break



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


Request Packet :  [6.0, 61.0, 13, 17.0, 41.0, 86, 33.0, 46.0, 66, 90.0, 27, 47, 39.0, 84.0, 29.0, 99.0, 76, 98, 57.0, 34.0, 100.0, 58, 31.0, 57.0, 18.0, 33]
Model Accuracy: 0.915625
a DDoS attack with accuracy percentage of :  0.915625



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


Request Packet :  [49.0, 98.0, 23, 28, 78.0, 94.0, 13, 31, 71, 49.0, 65, 65.0, 81, 19, 37.0, 1.0, 76, 83, 26, 33.0, 30, 51, 84, 30.0, 93, 50]
Model Accuracy: 0.915625
a DDoS attack with accuracy percentage of :  0.915625



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


Request Packet :  [54, 93, 71.0, 87, 44, 34.0, 42, 62.0, 17, 69.0, 51, 28, 94, 82.0, 34.0, 53, 91, 91, 67, 41, 70, 75, 80.0, 26, 20.0, 44.0]
Model Accuracy: 0.915625
a DDoS attack with accuracy percentage of :  0.915625
x
Packet Sending stopped:
